In [2]:
#Andrew Weis
#PPHA 30531 Data Skills for Public Policy
#PSet5
#26 November 2019

#Import packages
import pandas as pd
import bokeh
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
from ipywidgets import interact, interact_manual

from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [5]:
#File paths
PATH='/Users/andrewweis/Documents/GitHub/DataSkills/'
HW_FOLDER='assignment-5-awastro55'
DATA_FOLDER='chicago-police-data/data'
COMPLAINTS='/unified_data/complaints/'
DISCIPLINE='/context_data/'

In [25]:
#Generate the csv paths.
base_csv='complaints-{}_2000-2016_2016-11.csv'
datasets=['accused','victims','investigators','witnesses','complainants']
discipline_file='discipline_penalty_codes.csv'

def csv_gen(sets,base):
    return base.format(sets)

complaints_files=[csv_gen(item, base_csv) for item in datasets]

In [7]:
def read_data(csv):
    complaints_data=[]
    for item in csv:
        complaints_data.append(pd.read_csv(PATH+DATA_FOLDER+COMPLAINTS+item))
        
    return complaints_data

In [26]:
complaints_data=read_data(complaints_files)
discipline=pd.read_csv(PATH+DATA_FOLDER+DISCIPLINE+discipline_file)
#Now we have our data frames.  

In [9]:
#Find data types
def check(df):
    return df.dtypes

check(discipline)

CODE             int64
ACTION_TAKEN    object
NOTES           object
dtype: object

In [10]:
check(complaints_data[0])

row_id                                       int64
complaints-accused_2000-2016_2016-11_ID    float64
cr_id                                        int64
complaint_category                          object
recommended_discipline                     float64
final_discipline                           float64
recommended_finding                         object
final_finding                               object
UID                                          int64
old_UID                                      int64
link_UID                                   float64
dtype: object

In [11]:
check(complaints_data[1])

cr_id       int64
gender     object
age       float64
race       object
dtype: object

In [12]:
check(complaints_data[2])

row_id                                             int64
complaints-investigators_2000-2016_2016-11_ID      int64
cr_id                                              int64
first_name                                        object
last_name                                         object
middle_initial                                    object
suffix_name                                       object
appointed_date                                    object
current_star                                     float64
current_rank                                      object
current_unit                                     float64
UID                                              float64
old_UID                                          float64
link_UID                                         float64
dtype: object

In [13]:
check(complaints_data[3])

row_id                                         int64
complaints-witnesses_2000-2016_2016-11_ID    float64
cr_id                                          int64
UID                                          float64
link_UID                                     float64
dtype: object

In [27]:
check(complaints_data[4])

cr_id       int64
gender     object
age       float64
race       object
dtype: object

In [14]:
complaints_witnesses=complaints_data[3]

In [22]:
officer=PATH+DATA_FOLDER+COMPLAINTS+'officer-filed-complaints__2017-09.csv'
officer_df=pd.read_csv(officer)
officer

'/Users/andrewweis/Documents/GitHub/DataSkills/chicago-police-data/data/unified_data/complaints/officer-filed-complaints__2017-09.csv'

In [24]:
check(officer_df)

cr_id    int64
dtype: object

In [28]:
def new_column(df, original, new):
    df[new]=df[original]
    return df

In [30]:
#Copy columns in discipline frame to get ready for merging.  
discipline=new_column(discipline,'CODE','recommended_discipline')
discipline=new_column(discipline,'CODE','final_discipline')
discipline=new_column(discipline,'ACTION_TAKEN','recommended_action')
discipline=new_column(discipline,'ACTION_TAKEN','final_action')
#discipline

In [31]:
complaints_accused=complaints_data[0]
complaints_victims=complaints_data[1]
complaints_investigators=complaints_data[2]
complaints_complainants=complaints_data[4]

In [44]:
def merging():
    accrec=pd.merge(complaints_accused,discipline[['recommended_discipline','recommended_action']],
                    how='left',on='recommended_discipline')
    accnew=pd.merge(accrec,discipline[['final_discipline','final_action']], 
                how='left',on='final_discipline')
    accvic=pd.merge(accnew, complaints_data[1], how='outer', on='cr_id')
    acccom=pd.merge(accvic, complaints_data[4],how='outer', on='cr_id',
                   suffixes=('_vic', '_com'))
    accfinal=pd.merge(acccom,complaints_data[2][['cr_id', 'row_id',
                                             'complaints-investigators_2000-2016_2016-11_ID',
                                             'first_name','last_name',
                                             'middle_initial',
                                             'suffix_name',
                                             'appointed_date',
                                             'current_star',
                                             'current_rank',
                                             'current_unit']],how='outer',on='cr_id',
                     suffixes=('_acc','_inv'))
    return accfinal

In [45]:
accfinal=merging()
accfinal

,row_id_acc,complaints-accused_2000-2016_2016-11_ID,cr_id,complaint_category,recommended_discipline,final_discipline,recommended_finding,final_finding,UID,old_UID,...,row_id_inv,complaints-investigators_2000-2016_2016-11_ID,first_name,last_name,middle_initial,suffix_name,appointed_date,current_star,current_rank,current_unit
0,1.0,15646.0,258996,01A-USE OF PROFANITY,600.0,600.0,NS,NS,130754.0,1421.0,...,1.0,1.0,STEVEN,SCHWIEGER,NaN,NaN,1986-06-16,NaN,LIEUTENANT OF POLICE,13.0
1,2.0,8602.0,258997,01A-USE OF PROFANITY,600.0,600.0,UN,UN,117163.0,3556.0,...,2.0,2.0,MICHAEL,MULLIGAN,NaN,JR,1970-12-14,NaN,SERGEANT OF POLICE,8.0
2,3.0,6951.0,258997,01A-USE OF PROFANITY,600.0,600.0,UN,UN,113991.0,14292.0,...,2.0,2.0,MICHAEL,MULLIGAN,NaN,JR,1970-12-14,NaN,SERGEANT OF POLICE,8.0
3,4.0,13745.0,258998,10J-NEGLECT OF DUTY/CONDUCT UNBECOMING - ON DUTY,600.0,600.0,EX,EX,127168.0,26608.0,...,3.0,3.0,MAUREEN,MC MAHON,NaN,NaN,1985-07-01,NaN,LIEUTENANT OF POLICE,608.0
4,5.0,918.0,259001,10U-INADEQUATE/FAILURE TO PROVIDE SERVICE,600.0,600.0,UN,UN,101654.0,17539.0,...,6.0,6.0,CHARLES,POPIELARZ,NaN,NaN,1982-06-14,NaN,SERGEANT OF POLICE,1.0
5,6.0,10477.0,259001,10U-INADEQUATE/FAILURE TO PROVIDE SERVICE,600.0,600.0,UN,UN,120493.0,13708.0,...,6.0,6.0,CHARLES,POPIELARZ,NaN,NaN,1982-06-14,NaN,SERGEANT OF POLICE,1.0
6,7.0,2403.0,259002,05A-ARRESTEE - DURING ARREST,600.0,600.0,NS,NS,104455.0,3055.0,...,7.0,7.0,MONIKA,NORWOOD,NaN,NaN,1993-05-01,NaN,INVESTIGATOR 2 IPRA,113.0
7,8.0,8339.0,259002,05A-ARRESTEE - DURING ARREST,600.0,600.0,NS,NS,116784.0,19347.0,...,7.0,7.0,MONIKA,NORWOOD,NaN,NaN,1993-05-01,NaN,INVESTIGATOR 2 IPRA,113.0
8,9.0,14036.0,259005,07A-MISCONDUCT DURING ISSUANCE OF CITATION,600.0,600.0,NS,NS,127595.0,28345.0,...,9.0,9.0,BEATRICE,O'DONNELL,NaN,NaN,1987-04-13,NaN,CHIEF,120.0
9,10.0,5741.0,259008,09G-ABUSE OF AUTHORITY,600.0,600.0,UN,UN,111265.0,2829.0,...,12.0,12.0,THOMAS,RENNIE,NaN,NaN,1970-06-15,NaN,CAPTAIN OF POLICE,1.0


In [46]:
final_complaint=accfinal.drop_duplicates()
final_complaint

,row_id_acc,complaints-accused_2000-2016_2016-11_ID,cr_id,complaint_category,recommended_discipline,final_discipline,recommended_finding,final_finding,UID,old_UID,...,row_id_inv,complaints-investigators_2000-2016_2016-11_ID,first_name,last_name,middle_initial,suffix_name,appointed_date,current_star,current_rank,current_unit
0,1.0,15646.0,258996,01A-USE OF PROFANITY,600.0,600.0,NS,NS,130754.0,1421.0,...,1.0,1.0,STEVEN,SCHWIEGER,NaN,NaN,1986-06-16,NaN,LIEUTENANT OF POLICE,13.0
1,2.0,8602.0,258997,01A-USE OF PROFANITY,600.0,600.0,UN,UN,117163.0,3556.0,...,2.0,2.0,MICHAEL,MULLIGAN,NaN,JR,1970-12-14,NaN,SERGEANT OF POLICE,8.0
2,3.0,6951.0,258997,01A-USE OF PROFANITY,600.0,600.0,UN,UN,113991.0,14292.0,...,2.0,2.0,MICHAEL,MULLIGAN,NaN,JR,1970-12-14,NaN,SERGEANT OF POLICE,8.0
3,4.0,13745.0,258998,10J-NEGLECT OF DUTY/CONDUCT UNBECOMING - ON DUTY,600.0,600.0,EX,EX,127168.0,26608.0,...,3.0,3.0,MAUREEN,MC MAHON,NaN,NaN,1985-07-01,NaN,LIEUTENANT OF POLICE,608.0
4,5.0,918.0,259001,10U-INADEQUATE/FAILURE TO PROVIDE SERVICE,600.0,600.0,UN,UN,101654.0,17539.0,...,6.0,6.0,CHARLES,POPIELARZ,NaN,NaN,1982-06-14,NaN,SERGEANT OF POLICE,1.0
5,6.0,10477.0,259001,10U-INADEQUATE/FAILURE TO PROVIDE SERVICE,600.0,600.0,UN,UN,120493.0,13708.0,...,6.0,6.0,CHARLES,POPIELARZ,NaN,NaN,1982-06-14,NaN,SERGEANT OF POLICE,1.0
6,7.0,2403.0,259002,05A-ARRESTEE - DURING ARREST,600.0,600.0,NS,NS,104455.0,3055.0,...,7.0,7.0,MONIKA,NORWOOD,NaN,NaN,1993-05-01,NaN,INVESTIGATOR 2 IPRA,113.0
7,8.0,8339.0,259002,05A-ARRESTEE - DURING ARREST,600.0,600.0,NS,NS,116784.0,19347.0,...,7.0,7.0,MONIKA,NORWOOD,NaN,NaN,1993-05-01,NaN,INVESTIGATOR 2 IPRA,113.0
8,9.0,14036.0,259005,07A-MISCONDUCT DURING ISSUANCE OF CITATION,600.0,600.0,NS,NS,127595.0,28345.0,...,9.0,9.0,BEATRICE,O'DONNELL,NaN,NaN,1987-04-13,NaN,CHIEF,120.0
9,10.0,5741.0,259008,09G-ABUSE OF AUTHORITY,600.0,600.0,UN,UN,111265.0,2829.0,...,12.0,12.0,THOMAS,RENNIE,NaN,NaN,1970-06-15,NaN,CAPTAIN OF POLICE,1.0


In [42]:
by_crid=final_complaint.groupby(['complaint_category', 'final_finding']).nunique('cr_id')
by_crid

row_id_acc  \
complaint_category                                 final_finding               
01A-USE OF PROFANITY                               EX                     44   
                                                   NAF                    84   
                                                   NS                   2152   
                                                   SU                     62   
                                                   UN                    707   
01B-RACIAL/ETHNIC, ETC.                            EX                      3   
                                                   NAF                   263   
                                                   NS                    655   
                                                   SU                     36   
                                                   UN                    168   
01C-MISCELLANEOUS                                  EX                     25   
                                                   NAF                   111   
                                                   NS                    376   
                                                   SU                     12   
                                                   UN                    156   
02A-INTOXICATED ON DUTY                            NAF                     3   
                                                   NS                      5   
                                                   SU                     28   
                                                   UN                     22   
02B-INTOXICATED OFF DUTY                           EX                      1   
                                                   NAF                     3   
                                                   NS                     14   
                                                   SU                     54   
                                                   UN                     10   
02C-D.U.I. - ON DUTY                               NAF                     2   
                                                   UN                      2   
02D-D.U.I. - OFF DUTY                              NS                      2   
                                                   SU                    113   
                                                   UN                      3   
02E-POSSESSION/DRINKING ALCOHOL - ON DUTY          NAF                     4   
...                                                                      ...   
14B-FEDERAL CIVIL SUIT                             NS                    133   
                                                   SU                      4   
                                                   UN                    319   
15A-USE/ABUSE DRUGS/CONTR. SUBSTANCE - ON DUTY     NS                      1   
                                                   SU                      6   
                                                   UN                     15   
15B-USE/ABUSE DRUGS/CONTR. SUBSTANCE - OFF DUTY    NAF                     3   
                                                   NS                     10   
                                                   SU                      4   
                                                   UN                     35   
15C-D.U.I., DRUGS/ CONTR. SUB. - ON DUTY           NS                      1   
                                                   SU                      1   
                                                   UN                      2   
15D-D.U.I., DRUGS/ CONTR. SUB. - OFF DUTY          EX                      1   
                                                   NAF                     1   
                                                   SU                      3   
                                                   UN                      1   
15E-POSITIVE DRUG SCREEN - ORIGINATED FROM COMP... EX             

In [47]:
#by_race=final_complaint.groupby(['race', 'final_finding']).nunique('cr_id')
check(final_complaint)

row_id_acc                                       float64
complaints-accused_2000-2016_2016-11_ID          float64
cr_id                                              int64
complaint_category                                object
recommended_discipline                           float64
final_discipline                                 float64
recommended_finding                               object
final_finding                                     object
UID                                              float64
old_UID                                          float64
link_UID                                         float64
recommended_action                                object
final_action                                      object
gender_vic                                        object
age_vic                                          float64
race_vic                                          object
gender_com                                        object
age_com                        

In [54]:
plot = figure(title='Complaints by category', x_axis_label='Type', y_axis_label='Number')
plot.vbar(by_crid['complaint_category'], width=0.9, top=)
show(plot)

TypeError: hbar() missing 2 required positional arguments: 'height' and 'right'

In [ ]:
#I haven't been able to get the plots to work.  I wanted to make plots based on race, but
#grouping by race won't work, even though it's the same line of code as in the prior assignment.
#Just copied and pasted.  I also still couldn't figure out how to generalize the merging
#function.  In the spirit of observing the deadline, I'll submit my failed attempts. To
#improve this, I would generalize the merging function, create plotting function, and plot:
#number of complaints filed by people of each race.  Number of complaints sustained by people
#of each race.  Complaints sustained by category.  I will make further attempts to fix this
#code and may submit a correction if/when I achieve one. 